# Push Prices Handler

This module receives price changes from Module 2 or Module 3 and pushes them to the system.

## Required Input Columns
- `product_id`: Product ID
- `sku`: SKU
- `new_price`: New price to set
- `warehouse_id`: Warehouse ID
- `cohort_id`: Cohort ID

Only rows where `new_price` differs from `current_price` will be processed.

## Usage
Called from Module 2 and Module 3 after generating price recommendations.


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import pytz

# Cairo timezone
CAIRO_TZ = pytz.timezone('Africa/Cairo')
CAIRO_NOW = datetime.now(CAIRO_TZ)

print(f"Push Prices Handler loaded at {CAIRO_NOW.strftime('%Y-%m-%d %H:%M:%S')} Cairo time")


In [ ]:
def push_prices(df_prices: pd.DataFrame, source_module: str = 'unknown') -> dict:
    """
    Push price changes to the system.
    
    Parameters:
    -----------
    df_prices : pd.DataFrame
        DataFrame containing price data with required columns:
        - product_id: Product ID
        - sku: SKU
        - new_price: New price to set
        - warehouse_id: Warehouse ID
        - cohort_id: Cohort ID
        
    source_module : str
        Name of the calling module ('module_2' or 'module_3')
        
    Returns:
    --------
    dict with keys:
        - total_received: Total rows received
        - price_changes: Number of actual price changes
        - pushed: Number successfully pushed
        - failed: Number failed to push
        - skipped: Number skipped (no change)
    """
    
    result = {
        'total_received': len(df_prices),
        'price_changes': 0,
        'pushed': 0,
        'failed': 0,
        'skipped': 0,
        'source_module': source_module,
        'timestamp': datetime.now(CAIRO_TZ).strftime('%Y-%m-%d %H:%M:%S')
    }
    
    if df_prices.empty:
        print(f"⚠️ No data received from {source_module}")
        return result
    
    # Filter to only rows where price actually changed
    df_to_push = df_prices[
        (df_prices['new_price'].notna()) & 
        (df_prices['new_price'] != df_prices['current_price'])
    ].copy()
    
    result['price_changes'] = len(df_to_push)
    result['skipped'] = len(df_prices) - len(df_to_push)
    
    if df_to_push.empty:
        print(f"ℹ️ No price changes to push from {source_module}")
        return result
    
    print(f"\n{'='*60}")
    print(f"PUSH PRICES - Source: {source_module}")
    print(f"{'='*60}")
    print(f"Total received: {result['total_received']}")
    print(f"Price changes to push: {result['price_changes']}")
    print(f"Skipped (no change): {result['skipped']}")
    
    # Required columns for pushing
    required_cols = ['product_id', 'sku', 'new_price', 'warehouse_id', 'cohort_id']
    
    # Check for missing required columns
    missing_cols = [c for c in required_cols if c not in df_to_push.columns]
    if missing_cols:
        print(f"❌ ERROR: Missing required columns: {missing_cols}")
        result['failed'] = len(df_to_push)
        return result
    
    # Extract only the required columns
    push_data = df_to_push[required_cols].copy()
    
    print(f"\nPrice changes summary:")
    if 'current_price' in df_to_push.columns:
        print(f"  Increases: {len(df_to_push[df_to_push['new_price'] > df_to_push['current_price']])}")
        print(f"  Decreases: {len(df_to_push[df_to_push['new_price'] < df_to_push['current_price']])}")
    
    # ============================================================
    # TODO: Add actual price push logic here
    # The user will provide the API/method to push prices
    # ============================================================
    
    # Store the data to be pushed
    global prices_to_push
    prices_to_push = push_data
    
    print(f"\n📋 {len(push_data)} prices prepared for pushing")
    print(f"   Columns: {list(push_data.columns)}")
    print(f"   Data stored in 'prices_to_push' variable")
    
    # For now, mark all as "pushed" (placeholder)
    result['pushed'] = len(push_data)
    
    return result


# Store prices to push (accessible after calling push_prices)
prices_to_push = pd.DataFrame()
